In [1]:
!pip install --user tf-nightly

In [2]:
!git clone https://github.com/tensorflow/nmt

fatal: destination path 'nmt' already exists and is not an empty directory.


In [3]:
import tensorflow as tf
import matplotlib as plt
import os
import re
import string
import tensorboard as tb

from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, ReLU, RNN, LSTM, Embedding

C:\Users\jedgo\.conda\envs\fun\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jedgo\.conda\envs\fun\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\jedgo\.conda\envs\fun\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [4]:
# Embedding
embedding_encoder = variable_scope.get_variable(
    "embedding encoder", [src_vocab_size, embedding_size], ...)

# Look up the embedding
# encoder_inputs: [max_time, batch_size]
# encoder_emb_inp: [max_time, batch_size, embedding_size]
encoder_emb_inp = embedding_ops.embedding_lookup(
    embedding_encoder, encoder_inputs) 


NameError: name 'variable_scope' is not defined

In [7]:
# Build RNN Cell
encoder_cell = tf.keras.layers.LSTM(num_units)

# Run dynamic RNN
#   encoder_outputs: [max_time, batch_size, num_units]
#   encoder_state: [batch_size, num_units]

encoder_outputs, encoder_state = tf.keras.dynamic_rnn(
    encoder_cell, encoder_emb_inp,
    sequence_length=source_sequence_length, time_major=True)

NameError: name 'num_units' is not defined

In [ ]:
# Build RNN Cell
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

# Helper

helper = tf.contrib.seq2seq.TrainingHelper(
    decoder_emb_inp, decoder_lengths, time_major=-True)

# Decoder

decoder = tf.contrib.seq2seq.BasicDecoder(
    decoder_cell, helper, encoder_state,
    output_layer=projection_layer)

# Dynamic decoding

outputs, _ = tf.contrib.seq2seq.dynamic_decode(decode, ...)
logits = outputs.rnn_output

In [ ]:
projection_layer = layers_core.Dense(
    tgt_vocab_size, use_bias=False)

In [ ]:
# Loss

crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=decoder_outputs, logits=logits)
train_loss = (tf.reduce_sum(crossent * target_weights) / batch_size)

In [ ]:
# Calculate and Clip Gradients

params = tf.trainable_variables()
gradients = tf.gradients(train_loss, params)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradiant_norm)

# Optimization

optimizer = tf.train.AdamOptimizer(learning_rate)
update_step = optimizer.apply_gradients(zip(clipped_gradients, params))

In [ ]:
nmt/scripts/download_iwslt15.sh /tmp/nmt_data

In [ ]:
mkdir /tmp/nmt_model
python -m nmt.nmt \
    --src=vi --tgt=en \
    --vocab_prefix=/tmp/nmt_data/vocab  \
    --train_prefix=/tmp/nmt_data/train \
    --dev_prefix=/tmp/nmt_data/tst2012  \
    --test_prefix=/tmp/nmt_data/tst2013 \
    --out_dir=/tmp/nmt_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=2 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu

In [ ]:
`